## 비마트 센터별 주문 예측수

- 미래 3일치 시간대별 데이터 예측
- 일 1회 데이터 업데이트
- 데이터 제공 포멧: https://docs.google.com/spreadsheets/d/1XfrDtgyrS2h_aY0_6tL2BRBUith5drnnSLou6NUwAwg/edit#gid=0
- 용산한남점, 마포공덕점, 분당dms 2020/5/28에 오픈 (한달정도후에 모델에 반영)
- 인천남부 지점의 경우 2020/5/27~5/28 기록 삭제 필요 (영업점 이슈 발생)
- 참고: 센터별 운영시간
- 강남점 새벽 2시까지 / 관악점 22시30분까지(그외 센터는 9시부터 23시59분까지 운영함)

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from lightgbm import LGBMRegressor

mpl.rc("font", family="NanumGothic")
mpl.rcParams['axes.unicode_minus'] = False

import os
import pandas as pd
import numpy as np

os.environ['KMP_DUPLICATE_LIB_OK']='True'
pd.set_option('precision', 4)
np.set_printoptions(precision=3)

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-paper')

In [2]:
from pyhive import presto
import time

engine = presto.Connection(host="presto.david2.ds.woowa.in", port=12304, username="songhunhwa", source="ds_dev_tool-jupyter")

In [35]:
# export objects
## 평일인데 쉬는날 (선거일, 공휴일)
# holiday_list = ['2019-03-01', '2019-05-06', '2019-06-06', '2019-08-15', 
#                 '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', 
#                 '2019-10-09', '2019-12-25', '2020-01-01', '2020-01-24', 
#                 '2020-01-25', '2020-01-27', '2020-03-01', '2020-01-27', 
#                 '2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05', 
#                 '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-09', '2020-12-25']

# with open('../[비마트]센터별_주문수_예측모델/holiday_list.pickle', 'wb') as f:
#     pickle.dump(holiday_list, f, pickle.HIGHEST_PROTOCOL)

# lgb_reg = LGBMRegressor(objective='regression')
# joblib.dump(lgb_reg, '../[비마트]센터별_주문수_예측모델/lgb_reg_model.pickle', compress=1)

In [65]:
# import
from sklearn.externals import joblib
import pickle

with open('../[비마트]센터별_주문수_예측모델/holiday_list.pickle', 'rb') as f:
    holiday_list = pickle.load(f)

# 기본 모델
lgb_reg_model = joblib.load('../[비마트]센터별_주문수_예측모델/lgb_reg_model.pickle')

In [11]:
# 테이블 추출
query = """
        select
            part_date,
            created_date,
            ord.order_id 
            shop_id,
            shop_no,
            shop_nm
        from market.bm_order_summary ord

            inner join
            (
            select 
                shop_id,
                baemin_shop_no as shop_no,
                name as shop_nm,
                status as shop_status
            from market.bm_shop
            where address not like '%우아한형제도%'
                and status = 'ACTIVE'
            ) bm_shop
            on ord.shop_id = bm_shop.shop_id

        where part_date >= '2019-11-16'
            and part_date < '2020-06-22'
            and status = 'COMPLETED'
        """

In [12]:
df = pd.read_sql_query(query, engine)

In [13]:
df.sort_values("part_date").tail(10)

,part_date,created_date,shop_id,shop_no,shop_nm
2043750,2020-06-21,2020-06-21 13:16:11.000,5327374,13007548,B마트 영등포
2043752,2020-06-21,2020-06-21 13:16:24.000,5327383,13007548,B마트 영등포
2043753,2020-06-21,2020-06-21 13:16:35.000,5327392,13007548,B마트 영등포
2043754,2020-06-21,2020-06-21 13:16:36.000,5327393,13007548,B마트 영등포
2043755,2020-06-21,2020-06-21 13:16:51.000,5327406,13007548,B마트 영등포
2043756,2020-06-21,2020-06-21 13:16:54.000,5327411,10916477,B마트 강남
2043757,2020-06-21,2020-06-21 13:16:59.000,5327414,10916477,B마트 강남
2043758,2020-06-21,2020-06-21 13:17:17.000,5327423,10916477,B마트 강남
2043751,2020-06-21,2020-06-21 13:16:11.000,5327376,10916477,B마트 강남
1369564,2020-06-21,2020-06-21 12:38:11.000,5325311,13033820,B마트 중랑


In [32]:
main_center = df.shop_nm.value_counts()[:-6]

In [43]:
q2 = """select 
            shop_id, 
            name as shop_nm
        from market.bm_shop
        where address not like '%우아한형제도%'
            and status = 'ACTIVE'
     """                
                
center_id = pd.read_sql_query(q2, engine).set_index("shop_nm")            

In [51]:
pd.concat([center_id, main_center], axis=1).dropna()['shop_id']

B마트 강남       2
B마트 강동      28
B마트 강북      22
B마트 강서      26
B마트 관악      24
B마트 광진      20
B마트 금천      29
B마트 노원      21
B마트 마포       3
B마트 성동      27
B마트 송파       1
B마트 수원      32
B마트 영등포     10
B마트 은평      30
B마트 인천남부    31
B마트 중랑      23
Name: shop_id, dtype: object

In [15]:
from datetime import timedelta

today = pd.to_datetime('now')
start_dt = today.strftime("%Y-%m-%d")
end_dt = (today + timedelta(days=2)).strftime("%Y-%m-%d")

pred_start_dt = (today + timedelta(days=1)).strftime("%Y-%m-%d") + ' 9'
pred_end_dt_gang = (today + timedelta(days=1)).strftime("%Y-%m-%d") + ' 1'
pred_end_dt_nogang = (today + timedelta(days=1)).strftime("%Y-%m-%d") + ' 23'

# 관악지점 타임슬롯 조건 추가 9~22시
pred_end_dt_gwanak = (today + timedelta(days=1)).strftime("%Y-%m-%d") + ' 22'

pred_end_next_dt_gang = (today + timedelta(days=2)).strftime("%Y-%m-%d") + ' 1'
pred_end_next_dt_nogang = (today + timedelta(days=2)).strftime("%Y-%m-%d") + ' 23'

# 관악지점 타임슬롯 조건 추가 9~22시
pred_end_next_dt_gwanak = (today + timedelta(days=2)).strftime("%Y-%m-%d") + ' 22'

In [69]:
class BmartOrdPredModel():

    def __init__(self, holiday_list, reg_model):
        self._holiday_list = holiday_list
        self._reg_model = reg_model
            
    ## 불필요 데이터 제거
    def _drop_samples(self, df):
        # 운영이슈 (고정)
        idx_drop = df.query("shop_nm == 'B마트 인천남부'")\
                     .query("part_date >= '2020-05-27' and part_date <= '2020-05-28'").index
        df = df[~df.index.isin(idx_drop)]
        
        # 초기 오픈 지점 (가변적)
        new_open_rgn = ['B마트 마포공덕', 'B마트 분당', 'B마트 용산한남', 'B마트 일산']
        df = df[~df['shop_nm'].isin(new_open_rgn)]
        return df        
    
    # 날짜 포멧 변경    
    def _change_date_format(self, df):
        df['reg_date'] = pd.to_datetime(df['part_date'])
        df['reg_hour'] = pd.to_datetime(df['created_date']).dt.hour
        df.sort_values(['reg_date', 'reg_hour'], inplace=True)
        df.drop(['shop_id', 'part_date', 'created_date'], axis=1, inplace=True)   
        return df
    
    # 시간별로 집계
    def aggregate_hourly(self, df):                        
        df = self._drop_samples(df)
        df = self._change_date_format(df)        
        df = df.groupby(["shop_nm", "reg_date", "reg_hour"]).size().reset_index()
        df.columns = ['shop_nm', 'reg_date', 'reg_hour', 'ord_cnt']
        df['weekday'] = df['reg_date'].dt.weekday
        df['is_holiday'] = np.where((df['weekday'] == 5)|(df['weekday'] == 6)|df['reg_date'].isin(self._holiday_list), 1, 0)
        
        idx = (df['reg_date'].astype(str) + ' ' + df['reg_hour'].astype(str))
        return df.set_index(idx).drop('reg_date', axis=1)       
    
    # 예측기간 더미 데이터프레임 생성
    def _create_pred_period(self, rgn_nm, start_dt, end_dt):        
        pred_date = pd.date_range(start=start_dt, end=end_dt)

        if rgn_nm == 'B마트 강남':
            pred_hours = [i for i in range(9, 24)]
            pred_hours.extend([0,1])

        # 관악지점 타임슬롯 조건 추가 9~22시
        elif rgn_nm == 'B마트 관악':
            pred_hours = [i for i in range(9, 23)]    
        else:   
            pred_hours = [i for i in range(9, 24)]

        pred_date_list = []
        pred_hour_list = []
        for day in pred_date:
            for hour in pred_hours:
                pred_date_list.append(day)
                pred_hour_list.append(hour)

        # 추출 테이블 및 예측 더미기간 병합
        pred_df = pd.concat([pd.Series(pred_date_list), pd.Series(pred_hour_list)], axis=1)
        pred_df.columns=['reg_dt', 'reg_hour']
        pred_df['shop_nm'] = rgn_nm
        pred_df['reg_hour_idx'] = pred_df['reg_hour']
        pred_df['weekday'] = pred_df['reg_dt'].dt.weekday
        pred_df['is_holiday'] = np.where((pred_df['weekday'] == 5) | (pred_df['weekday'] == 6) | pred_df['reg_dt'].isin(holiday_list), 1, 0)
        pred_df['ord_cnt'] = np.nan
        
        idx = (pred_df['reg_dt'].astype(str) + ' ' + pred_df['reg_hour_idx'].astype(str))
        return pred_df.set_index(idx).drop(['reg_dt', 'reg_hour_idx'], axis=1)

    # fitting
    def _fit_predict(self, df, start_dt):
        train = df[df.index < start_dt]
        test = df[df.index >= start_dt]
        train_X = train.drop(['shop_nm', 'ord_cnt'], axis=1)
        train_y = train['ord_cnt']
        test_X = test.drop(['shop_nm', 'ord_cnt'], axis=1).dropna()
  
        self._reg_model.fit(train_X, train_y)
        return self._reg_model.predict(test_X).round(0).astype(int)        
    
    # 지점별 반복 예측 
    def predict(self, start_dt, end_dt):
        rgn_list = df_agg.shop_nm.drop_duplicates().values
        pred_df_final = pd.DataFrame() 
        
        for rgn_nm in rgn_list: 
            df_agg_rgn = df_agg.query("shop_nm == @rgn_nm")
  
            # 예측기간 더미
            pred_period_rgn = self._create_pred_period(rgn_nm, start_dt, end_dt)

            # 병합 및 이전시간대 데이터 feature 추가
            df_rgn = pd.concat([df_agg_rgn, pred_period_rgn])     
            df_rgn['ord_cnt_prev_week'] = df_rgn.groupby(["weekday", "reg_hour"])['ord_cnt'].shift(1)
            df_rgn['ord_cnt_prev_day'] = df_rgn.groupby("reg_hour")['ord_cnt'].shift(1)
            df_rgn['is_holiday_prev_day'] = df_rgn.groupby("reg_hour")['is_holiday'].shift(1)

            # 1차 타임 예측        
            pred = self._fit_predict(df_rgn, start_dt)
            if rgn_nm == 'B마트 강남':
                df_rgn.loc[pred_start_dt:pred_end_dt_gang,'ord_cnt_prev_day'] = pred.round(0).astype(int)
            
            # 관악지점 타임슬롯 조건 추가 9~22시
            elif rgn_nm == 'B마트 관악':
                df_rgn.loc[pred_start_dt:pred_end_dt_gwanak,'ord_cnt_prev_day'] = pred.round(0).astype(int)
            else:
                df_rgn.loc[pred_start_dt:pred_end_dt_nogang,'ord_cnt_prev_day'] = pred.round(0).astype(int)

            # 2차 타임 예측
            pred = self._fit_predict(df_rgn, start_dt)
            if rgn_nm == 'B마트 강남':
                df_rgn.loc[pred_start_dt:pred_end_next_dt_gang,'ord_cnt_prev_day'] = pred.round(0).astype(int)
            
            # 관악지점 타임슬롯 조건 추가 9~22시
            elif rgn_nm == 'B마트 관악':
                df_rgn.loc[pred_start_dt:pred_end_next_dt_gwanak,'ord_cnt_prev_day'] = pred.round(0).astype(int)
            else:
                df_rgn.loc[pred_start_dt:pred_end_next_dt_nogang,'ord_cnt_prev_day'] = pred.round(0).astype(int)

            # 마지막 예측
            pred_final = self._fit_predict(df_rgn, start_dt)
            df_rgn_pred = df_rgn[df_rgn.index >= start_dt]
            df_rgn_pred['ord_cnt'] = pred_final
        
            pred_df_rgn = df_rgn_pred[['shop_nm', 'ord_cnt']]
            pred_df_final = pred_df_final.append(pred_df_rgn)

        return pred_df_final
            
#             # 추가 2020.6.15
#             pred_df_final.reset_index(inplace=True)
#             pred_df_final['pred_date'] = pred_df_final['index'].str.split(" ").str[0]
#             pred_df_final['pred_hour'] = pred_df_final['index'].str.split(" ").str[1]
#             pred_df_final.drop("index", axis=1)
            #pred_df_final[['pred_date', 'pred_hour', 'shop_nm', 'ord_cnt']].rename(columns={"ord_cnt":"pred_ord_cnt"})

In [70]:
%%time
# 객체생성
bmart_order_pred = BmartOrdPredModel(holiday_list, lgb_reg_model)

# 전처리 및 시간대별 집계
df_agg = bmart_order_pred.aggregate_hourly(df)

# 지점별 예측값 처리
pred_df_final = bmart_order_pred.predict(start_dt, end_dt)

CPU times: user 8min 17s, sys: 3.35 s, total: 8min 20s
Wall time: 30.9 s


In [71]:
# 2020.6.15 추가
pred_df_final1 = pred_df_final.reset_index()

pred_df_final1['pred_date'] = pred_df_final1['index'].str.split(" ").str[0]
pred_df_final1['pred_hour'] = pred_df_final1['index'].str.split(" ").str[1]

pred_df_final2 = pred_df_final1.drop("index", axis=1)
pred_df_final2 = pred_df_final2[['shop_nm','pred_date', 'pred_hour', 'ord_cnt']].rename(columns={"ord_cnt":"pred_ord_cnt"})

pred_df_final2

,shop_nm,pred_date,pred_hour,pred_ord_cnt
0,B마트 강남,2020-06-17,9,124
1,B마트 강남,2020-06-17,10,139
2,B마트 강남,2020-06-17,11,153
3,B마트 강남,2020-06-17,12,141
4,B마트 강남,2020-06-17,13,138
5,B마트 강남,2020-06-17,14,164
6,B마트 강남,2020-06-17,15,157
7,B마트 강남,2020-06-17,16,172
8,B마트 강남,2020-06-17,17,168
9,B마트 강남,2020-06-17,18,204


In [72]:
# export
pred_df_final2.to_excel("pred_res_20200617.xlsx")